# check output files

In [ ]:
import numpy as np
import numpy.matlib
import numpy.ma as ma

import xarray as xr
#xr.set_options(enable_cftimeindex=True)
#from xarray.coding.times import CFTimedeltaCoder
time_coder = xr.coders.CFDatetimeCoder(use_cftime=True) #create time coder with cftime

import time
import cftime
import netCDF4 as nc
from datetime import timedelta

import pandas as pd

#import xcdat

import matplotlib as mpl
import matplotlib.pyplot as plt

#import cartopy.crs as ccrs
#from cartopy.util import add_cyclic_point

import glob


## notes on packages to add to this kernel
import nc_time_axis
import cartopy.crs as ccrs
import cartopy

 float m01s19i013(pseudo_level, latitude, longitude) ;    =  Surface type fractions

                int latitude_longitude ;
                int pseudo_level(pseudo_level) ;
                float latitude(latitude) ;
                float longitude(longitude) ;
                double forecast_period ;
                double forecast_period_bnds(bnds) ;
                double forecast_reference_time ;
                double time ;
                double time_bnds(bnds) ;
                float m01s19i042(latitude, longitude) ;    LANDUSE CO2 FLUX TO ATM    KGC/M2/YR
                float m01s19i044(latitude, longitude) ;    HARVEST C (GBM) KGC/M2/360d
                float m01s19i053(latitude, longitude) ;    SOIL TO ATMOS RESP FLUX  KGC/M2/YR
                float m01s19i102(latitude, longitude) ;    NPP POST N-LIMIT (GBM) KGC/M2/YR
                float m01s30i464(latitude_0, longitude_0) ;   ATMOSPHERIC CO2 BURDEN IN GtC
                float latitude_0(latitude_0) ;
                float longitude_0(longitude_0) ;
                float m01s30i466(latitude_0, longitude_0) ;    CO2 CONCENTRATION (ppmv)
                float m01s00i250(latitude, longitude) ;       CO2 OCEAN FLUX             KG/M**2/S
                float air_temperature(latitude, longitude) ;
                double height ;
                float air_temperature_0(latitude, longitude) ;
                float air_temperature_1(dim0, latitude, longitude) ;
                double forecast_period_0(dim0) ;
                double forecast_period_0_bnds(dim0, bnds) ;
                double time_0(dim0) ;
                double time_0_bnds(dim0, bnds) ;
                float soil_carbon_content(latitude, longitude) ;
                float toa_outgoing_longwave_flux(latitude, longitude) ;
                float toa_outgoing_longwave_flux_0(dim0, latitude, longitude) ;
                float toa_outgoing_shortwave_flux(latitude, longitude) ;
                float toa_outgoing_shortwave_flux_0(dim0, latitude, longitude) ;
                float vegetation_carbon_content(latitude, longitude) ;

In [ ]:
outputdir= '/glade/campaign/cgd/tss/people/aswann/flat10/'
filenamepath = outputdir +'/testing/' +'dh493a.py21711201.nc'

In [ ]:
ds_r = dsmerge_f = xr.open_dataset(filenamepath, use_cftime=True, mask_and_scale=True)


In [ ]:
#ds_masked = ds_r.where(ds_r >1e35)  

In [ ]:
ds_r

In [ ]:
ds_r.soil_carbon_content.where(ds_r.soil_carbon_content <1e35) .plot()

In [ ]:
ds_r.vegetation_carbon_content.where(ds_r.soil_carbon_content <1e35) .plot()

In [ ]:
outputdir= '/glade/campaign/cgd/tss/people/aswann/flat10/'

model='ACCESS-ESM1-5'
run='flat10_cdr'
var='cVeg'

searchpath= outputdir +model +'/' +run +'/*' +var +'*.nc'
filenamelist= np.sort(glob.glob(searchpath))


print(filenamelist)

#ds_access=


In [ ]:
############

In [ ]:
# load custom functions for analyzing flat10

from loading_function_flat10 import load_flat10, load_grid, select_time_slice, weighted_temporal_mean 

In [ ]:
data_dict={}

In [ ]:
modellist_t= ['ACCESS-ESM1-5',  
            'CESM2',    
            'GFDL-ESM4',  
            'GISS_E2.1',  
            'NorESM2-LM',
            'MPI-ESM1-2-LR',
            'CNRM-ESM2-1',
            'HadCM3LC-Bris']

runlist = ['flat10','flat10_zec','flat10_cdr']
# use a wildcard to capture different ways the folders and runs are named across models
runlist_wc = ['*lat10','*zec','*cdr']

varlist=['tas']

data_dict=load_flat10(data_dict, modellist_t, runlist, runlist_wc, varlist)

In [ ]:
data_dict={}

In [ ]:
outputdir= '/glade/campaign/cgd/tss/people/aswann/flat10/'
model='NorESM2-LM'
run='flat10_zec'
var='cSoil'

# single file
#filename='cSoil_Lmon_NorESM2-LM_flat10-zec_r1i1p1f1_gn_185002-185912.nc'

#outputdir +model +'/' +run +'/*' +var +'_*.nc'

#searchpath = outputdir +model +'/' +run +'/cSoil*.nc'
searchpath= outputdir +model +'/' +run +'/*' +var +'_*.nc'

filenamelist= np.sort(glob.glob(searchpath))

In [ ]:
filenamelist

In [ ]:
ds=xr.open_dataset('/glade/campaign/cgd/tss/people/aswann/flat10/NorESM2-LM/flat10_zec/cSoil_Emon_NorESM2-LM_flat10_zec_r1i1p1f1_gn_196001-196912.nc', decode_times=time_coder)

In [ ]:
ds['cSoil'].mean(dim='lon').plot()

In [ ]:
# load custom functions for analyzing flat10

from loading_function_flat10 import load_flat10, load_grid, select_time_slice, weighted_temporal_mean 

modellist=['NorESM2-LM']

In [ ]:
# load grid
data_dict = load_grid(data_dict,modellist)

In [ ]:
ds_area = data_dict[model +'_' +'areacella']
ds_landfrac = data_dict[model +'_' +'landfrac']
#area = ds_area['areacella'].expand_dims(dim={'time': ds.time.size}, axis=0)
#landfrac=ds_landfrac['sftlf'].expand_dims(dim={'time': ds.time.size}, axis=0)

area = ds_area['areacella'].reindex_like(ds, method='nearest',tolerance=0.05)

landfrac=ds_landfrac['sftlf'].reindex_like(ds, method='nearest',tolerance=0.05)

if landfrac.max(dim=['lat','lon'])>1: #test if landfrac is on a 0-100 or 0-1 scale
    landfrac=landfrac/100
    
landarea=area*landfrac


landarea

In [ ]:
landarea.plot()

In [ ]:
csoil=ds.cSoil
csoil

In [ ]:
csoil.sum(dim=['lat','lon']).plot()



In [ ]:
C_global =(csoil*landarea).sum(dim=['lat','lon'])

In [ ]:
C_global.plot()

# Zonal correction for NorESM

In [ ]:
# load custom functions for analyzing flat10

from loading_function_flat10 import load_flat10, load_grid, select_time_slice, weighted_temporal_mean 

In [ ]:
outputdir= '/glade/campaign/cgd/tss/people/aswann/flat10/'

modellist_orig= ['ACCESS-ESM1-5',  
            'CESM2',    
            'GFDL-ESM4',  
            'GISS_E2.1',  
            'NorESM2-LM',
            'MPI-ESM1-2-LR',
            'CNRM-ESM2-1',
            'HadCM3LC-Bris']


runlist = ['flat10','flat10_zec','flat10_cdr']
# use a wildcard to capture different ways the folders and runs are named across models
runlist_wc = ['*lat10','*zec','*cdr']

varlist_load=['cVeg','cSoil','cLitter','nbp','gpp','rh'] #, 'gpp','fgco2', 'ra', 'rh']#, 'npp'] # not working beyond nbp for norESM
varlist_analyze=['cVeg','cSoil','cTot','cLitter','nbp','gpp','rh']
varlist=varlist_load
unitslist=['kgC m-2','kgC m-2','kgC m-2','kgC m-2 s-1','kgC m-2 s-1','kgC m-2 s-1']

# there seems to be a problem with ra for NorESM

modelcolors=['tab:blue','tab:orange','tab:green','tab:red','tab:gray','tab:purple','tab:cyan','gold','tab:brown']
### from ben: colors=["tab:cyan","tab:olive","tab:green","tab:red","tab:gray","tab:pink","limegreen","tab:brown", "slateblue","gold"]

In [ ]:
modellist=['NorESM2-LM']

In [ ]:
# initialize a dictionary to hold all of the data
data_dict={}

In [ ]:
# load grid
data_dict = load_grid(data_dict,modellist)

In [ ]:
# load all data
data_dict=load_flat10(data_dict, modellist, runlist, runlist_wc, varlist)

### First lets try the 2d fields from Jerry Tjiputra

In [ ]:
import pickle

In [ ]:
## THESE BOTH REQUIRE A RECENT VERSION OF NUMPY.... 
# must use NPL 2025b (not a 2023 version)

totveg_corr=pd.read_pickle('NorESM2-LM_2D_TOTVEGC_ann_drift.pkl') 

field = pickle.load(open('NorESM2-LM_2D_TOTVEGC_ann_drift.pkl','rb'))

# units are  [g C m-2 yr-1] 

In [ ]:
totveg_corr

In [ ]:
field

In [ ]:
np.shape(np.squeeze(field))

In [ ]:
m=0

ds0 = data_dict[modellist[m] +'_' +runlist[0]] #flat10
ds1 = data_dict[modellist[m] +'_' +runlist[1]] #flat10-zec
ds2 = data_dict[modellist[m] +'_' +runlist[2]] #flat10-cdr

In [ ]:
# make xarray dataarrays to make it easier to apply to time dimensions

# for flat10, starts at 1850
adj_matrix = xr.DataArray(np.squeeze(field), dims=['lat','lon'], coords={'latitude': ds0.lat, 'longitude':ds0.lon})##,unit={'g C m-2 yr-1'})


In [ ]:
ty=ds0['time'].dt.year


In [ ]:
ty

In [ ]:

tyindx=ty-ty[0]+1


In [ ]:
tyindx

In [ ]:

ds0_adj = adj_matrix* tyindx


In [ ]:
ds0_adj

In [ ]:
# make xarray dataarrays to make it easier to apply to time dimensions

# for flat10, starts at 1850
adj_matrix = xr.DataArray(np.squeeze(field), dims=['lat','lon'], coords={'lat': ds0.lat, 'lon':ds0.lon})##,unit={'g C m-2 yr-1'})
##time_vector = xr.DataArray(np.arange(len(ds0.time)), dims='time', coords={'time': ds0.time})
ty=ds0['time'].dt.year
tyindx=ty-ty[0]+1
#time_vector = xr.DataArray(tyindx, dims='time', coords={'time': ds0.time})

ds0_adj = adj_matrix* tyindx


ds0_adj

In [ ]:
#arg=ds0.time.year
arg=ds0['time'].dt.year

arg-arg[0]+1

In [ ]:
ds0_adj.mean(dim='lon').plot()

In [ ]:
ds1.mean(dim='lon').plot()

### Zonal files

In [ ]:
import pickle

In [ ]:
# load zonal corrections for NorESM

field = pickle.load(open('NorESM2-LM_zonally_drift_Veg_Soil_Litt_C_1850-2199.dat', 'rb'))   
lat   = field[0] #latitude  
tr_veg = field[1] #trend in cumulative vegetation C [PgC/yr]  
tr_soi = field[2] #trend in cumulative soil C [PgC/yr]  
tr_lit = field[3] #trend in cumulative litter C [PgC/yr]  
tr_veg_avg = field[4] #trend in averaged vegetation C [gC/m2/yr]  
tr_soi_avg = field[5] #trend in averaged soil C [gC/m2/yr]  
tr_lit_avg = field[6] #trend in averaged litter C [gC/m2/yr]  


In [ ]:
m=0

ds0 = data_dict[modellist[m] +'_' +runlist[0]] #flat10
ds1 = data_dict[modellist[m] +'_' +runlist[1]] #flat10-zec
ds2 = data_dict[modellist[m] +'_' +runlist[2]] #flat10-cdr

In [ ]:
# make xarray vectors to make it easier to apply to time dimensions

# for flat10, starts at 1850
adj_vector = xr.DataArray(tr_veg, dims='lat', coords={'latitude': ds0.lat})
time_vector = xr.DataArray(np.arange(len(ds0.time)), dims='time', coords={'time': ds0.time})

ds0_adj = adj_vector * time_vector

ds0_adj.plot()

In [ ]:
# the adjustment needs to be different for the other experiments since they start later in time

# for flat10-zec, starts at 1950
adj_vector = xr.DataArray(tr_veg, dims='lat', coords={'latitude': ds0.lat})
time_vector = xr.DataArray(np.arange(len(ds1.time)), dims='time', coords={'time': ds1.time})

start_year = 1850

years_since_start = xr.DataArray(
    ds1['time'].dt.year - start_year,
    coords={'time': ds1['time']},
    dims='time'
)


# Broadcast to match the shape of data (time, lat)
adjustment = years_since_start * adj_vector


# Remove the trend
ds1_adj = ds1.cVeg.mean(dim='lon') - adjustment


adjustment.plot()

In [ ]:


ds1_adj.plot()

In [ ]:
def adj_NorESM_zonal(dsz):

    """
    adjusts the zonal mean profile in dsz by trend_zonal
    specific for files sent by NorESM
    """
    # for flat10-zec, time series starts at 1950
    time_vector = xr.DataArray(np.arange(len(dsz.time)), dims='time', coords={'time': dsz.time})
    
    start_year = 1850
    
    years_since_start = xr.DataArray(
        dsz['time'].dt.year - start_year,
        coords={'time': dsz['time']},
        dims='time'
    )

    # load trends to apply, load zonal corrections for NorESM
    import pickle
    field = pickle.load(open('NorESM2-LM_zonally_drift_Veg_Soil_Litt_C_1850-2199.dat', 'rb'))   
    lat   = field[0] #latitude  
    tr_veg = field[1] #trend in cumulative vegetation C [PgC/yr]  
    tr_soi = field[2] #trend in cumulative soil C [PgC/yr]  
    tr_lit = field[3] #trend in cumulative litter C [PgC/yr]  
    tr_veg_avg = field[4] #trend in averaged vegetation C [gC/m2/yr]  
    tr_soi_avg = field[5] #trend in averaged soil C [gC/m2/yr]  
    tr_lit_avg = field[6] #trend in averaged litter C [gC/m2/yr]  

    adj_vector = xr.DataArray(trend_zonal, dims='lat', coords={'latitude': dsz.lat})
    
    # Broadcast to match the shape of data (time, lat)
    adjustment = years_since_start * adj_vector

    
    # Remove the trend
    dsz_adj[var] = dsz[var].mean(dim='lon') - adjustment # this needs to either be a new entry in dsz or a new dataset with all variables?

    return dsz





In [ ]:
ds1_adj=adj_NorESM_zonal(dz1,

In [ ]:

# Inputs
# Example trend: 0.02 units per year
trend = 0.02  
start_year = 2000  # Year to treat as baseline

# Assume `data` is your xarray.DataArray with dims ("time", "lat")
# Convert time to years since start_year
years_since_start = xr.DataArray(
    data['time'].dt.year - start_year,
    coords={'time': data['time']},
    dims='time'
)

# Expand years_since_start to match shape of data
# So it can broadcast across the 'lat' dimension
adjustment = trend * years_since_start
adjustment = adjustment.broadcast_like(data)

# Remove the trend
detrended = data - adjustment


###
import xarray as xr

# Inputs:
# - data: DataArray with dims ("time", "lat")
# - trend: DataArray with dims ("lat",), one trend per latitude
# - start_year: e.g., 2000

# Compute years since the start year
years_since_start = xr.DataArray(
    data['time'].dt.year - start_year,
    coords={'time': data['time']},
    dims='time'
)

# Broadcast to match the shape of data (time, lat)
adjustment = years_since_start * trend

# Remove the trend
detrended = data - adjustment



# Loading UKESM

In [ ]:
modellist=['UKESM1.2']

In [ ]:
#---- get grid info

# loop over models
for m in range(len(modellist)):
    model=modellist[m]

    print(model +' getting grid info')
    # get land fraction
    filenamelist= glob.glob(outputdir +model +'/*/*sftlf*.nc')
    landfrac = xr.open_dataset(filenamelist[0], use_cftime=True)

    # get area of gridcells
    filenamelist= glob.glob(outputdir +model +'/*/*areacella*.nc')
    areacella = xr.open_dataset(filenamelist[0], use_cftime=True)

    ## get area of ocean gridcells
    #filenamelist= glob.glob(outputdir +model +'/*/*areacello*.nc')
    #areacello = xr.open_dataset(filenamelist[0], use_cftime=True)
    #if model =='CESM2':
    #    areacello=areacello*1e-4 # CESM2 has area units of cm2 for ocean

    #----correct the name of the lat lon dimensions for landfrac and areacella
    if ((model =='HadCM3LC-Bris' or 'UKESM1.2') and ('lat' not in landfrac)):
        #-- change latitude and longitude to lat and lon for HadCM3
        landfrac = landfrac.rename({'longitude': 'lon','latitude': 'lat'})
        areacella = areacella.rename({'longitude': 'lon','latitude': 'lat'})
        #-- change name of area fields to match other models
        areacella = areacella.rename({'cell_area': 'areacella'})
        landfrac = landfrac.rename({'land_area_fraction': 'sftlf'})

    if (model =='GISS_E2.1'):
        # lon is -180 to 180 in data but 0 to 360 in grid files =>convert
        areacella['lon']=areacella['lon']-180
        landfrac['lon']=landfrac['lon']-180
        #landfrac['lon']=landfrac['lon']-180
        #landfrac.reindex_like(areacella, method='nearest',tolerance=0.05)
        
    # add to the dictionary
    data_dict[model +'_areacella'] = areacella
    data_dict[model +'_landfrac'] = landfrac
    #data_dict[model +'_areacello'] = areacello

In [ ]:
#modellist = ['NorESM2-LM']

In [ ]:
# this loads all variables in varlist for all experiments in runlist and models in modellist

#modellist=['GFDL-ESM4',  'GISS_E2.1',  'NorESM2-LM','MPI-ESM1-2-LR']
#modellist=['HadCM3LC-Bris']
#----loop over models----#

for m in range(len(modellist)):
#for m in range(len(['GFDL-ESM4',  'GISS_E2.1',  'NorESM2-LM','MPI-ESM1-2-LR'])):
    model=modellist[m]
    print('loading model: ' +model)
    #----loop over experiments----# 
    for r in range(len(runlist)):
        run = runlist_wc[r]
        print('loading run: ' +run)
        #----loop over variables----#
        for v in range(len(varlist)):
            var=varlist_load[v]
            print('loading variable: ' +var)
            
            searchpath= outputdir +model +'/' +run +'/*' +var +'_*.nc'
            if ((model =='CESM2') or (model == 'UKESM1.2')):
                # all models have a var_ filename except CESM
                searchpath= outputdir +model +'/' +run +'/*' +var +'*.nc'
            
            filenamelist= np.sort(glob.glob(searchpath)) # sort in time order, xarray was having trouble arranging some of them in time dim

            #----loop over filenames----#
            # some variables are stored in multiple files
            # this should be possible with xr.open_mfdataset but it isn't loading all of time points
            for f in range(len(filenamelist)):
                file = filenamelist[f]
                if f==0:
                    dsmerge_f = xr.open_dataset(file, use_cftime=True)
                else:
                    ds = xr.open_dataset(file, use_cftime=True)
                    dsmerge_f=xr.concat([dsmerge_f,ds],dim='time')

            #----- Dealing with GISS----#
            # GISS does not have a "time" index, and instead just a list of years
            # lets replace the "year" dimension (data is called "years")
            # with a cftime object called "time" so it matches the other models
            # some variables don't have the variable that defines years at all
            if ((model == 'GISS_E2.1') and ('time' not in dsmerge_f)):         
                if 'year' in dsmerge_f: # if it has a variable called year, use that to make the time index
                    time_index = [cftime.DatetimeNoLeap(year, 1, 1) for year in dsmerge_f.year]
                else: # if it does not have a variable for year, use the size of the year dimension to make the time index
                    startyear=[1850, 1950, 1950] # these are the start years for each experiment for GISS
                    years = np.arange(startyear[r], startyear[r]+len(dsmerge_f['year']))
                    time_index = [cftime.DatetimeNoLeap(year, 1, 1) for year in years]
                
                # Create a new DataArray with cftime objects
                time_da = xr.DataArray(time_index, dims='year')
                # Add time_da as a coordinate to the dataset
                dsmerge_f.coords['time'] = time_da
                # Now, swap dimensions from 'years' to 'time'
                dsmerge_f = dsmerge_f.swap_dims({'year': 'time'})
                # drop the year variable
                #dsmerge_f = dsmerge_f.drop_vars('year')
            
            #----correct the name of the lat lon dimensions
            if ((model =='HadCM3LC-Bris') or (model == 'UKESM1.2') and ('lat' not in dsmerge_f)):
                #-- change latitude and longitude to lat and lon for HadCM3
                dsmerge_f = dsmerge_f.rename({'longitude': 'lon','latitude': 'lat'})
            
            #----correct variable names----# 
            if 'nep' in dsmerge_f: # one model has nbp called nep instead -> add an nbp variable that is a copy of nep
                dsmerge_f['nbp'] = dsmerge_f['nep']
                #dsmerge_f = dsmerge_f.drop_vars('nep') # to remove it from the dataset
            
            if model =='HadCM3LC-Bris':
                if 'GBMVegCarb_srf' in dsmerge_f: #HadCM3 
                    dsmerge_f['cVeg'] = dsmerge_f['GBMVegCarb_srf']
                if 'soilCarbon_srf' in dsmerge_f: #HadCM3 
                    dsmerge_f['cSoil'] = dsmerge_f['soilCarbon_srf']
                if 'NPP_mm_srf' in dsmerge_f: #HadCM3 
                    dsmerge_f['npp'] = dsmerge_f['NPP_mm_srf']
                if 'unknown' in dsmerge_f: #HadCM3 
                    dsmerge_f['nbp'] = dsmerge_f['unknown']
                if 'field1560_mm_srf' in dsmerge_f: #HadCM3 
                    dsmerge_f['fgco2'] = dsmerge_f['field1560_mm_srf']
                if 'soilResp_mm_srf' in dsmerge_f: #HadCM3 cSoil
                    dsmerge_f['rh'] = dsmerge_f['soilResp_mm_srf']
                if 'GPP_mm_srf gpp' in dsmerge_f: #HadCM3 cSoil
                    dsmerge_f['gpp'] = dsmerge_f['GPP_mm_srf gpp']
            
            if model =='UKESM1.2':
                missing_value = 1.0e36
                for var_name, vari in dsmerge_f.data_vars.items():
                    # Apply only if variable is numeric and has at least one dimension
                    if np.issubdtype(vari.dtype, np.number):
                        dsmerge_f[var_name] = vari.where(vari < missing_value * 0.1, np.nan)
                if 'vegetation_carbon_content' in dsmerge_f: #UKESM 
                    dsmerge_f['cVeg'] = dsmerge_f['vegetation_carbon_content']
                if 'soil_carbon_content' in dsmerge_f: #UKESM
                    dsmerge_f['cSoil'] = dsmerge_f['soil_carbon_content']
                if 'm01s19i102' in dsmerge_f: #UKESM 
                    dsmerge_f['npp'] = dsmerge_f['m01s19i102']
                #if 'unknown' in dsmerge_f: #UKESM  ###I CANT FIND NBP, will need to be constructed from soil resp, plant resp, and gpp? should verify with UKESM group
                #    dsmerge_f['nbp'] = dsmerge_f['unknown']
                if 'm01s00i250' in dsmerge_f: #UKESM
                    dsmerge_f['fgco2'] = dsmerge_f['m01s00i250']
                if 'm01s19i053' in dsmerge_f: #UKESM
                    dsmerge_f['rh'] = dsmerge_f['m01s19i053']
                if 'm01s19i183' in dsmerge_f: #UKESM   
                    dsmerge_f['gpp'] = dsmerge_f['m01s19i183']


            
            #----check units and convert if necessary----#
            if var in dsmerge_f: 
                if model =='CESM2':
                    if dsmerge_f[var].units == 'gC/m^2/s':
                        dsmerge_f[var]=dsmerge_f[var]*(1/1000) # convert from gC to kgC
                        dsmerge_f[var].attrs['units'] = 'kg m-2 s-1'
                    # stock variables
                    elif dsmerge_f[var].units == 'gC/m^2':
                        dsmerge_f[var]=dsmerge_f[var]*(1/1000) # convert from gC to kgC
                        dsmerge_f[var].attrs['units'] = 'kg m-2'

                # the units for cVeg in GISS look like they MUST be in gC rather than kgC 
                # CHANGING THE UNIT - even though it is reported as kgC, assuming it is in gC
                if ((var == 'cVeg') and (model == 'GISS_E2.1')):
                    dsmerge_f[var]=dsmerge_f[var]*(1/1000) # convert from gC to kgC

                
            else: #var does not exist
                ds=dsmerge_f
                # add a blank variable so that loops work
                if 'time' in ds:
                    nan_dataarray = xr.DataArray(np.full((len(ds['time']),len(ds['lat']), len(ds['lon'])), np.nan), 
                                                 coords={'lon': ds['lon'], 'lat': ds['lat'],'time': ds['time']}, dims=['time','lat', 'lon'])
                #else: # this should now be obsolete
                #    nan_dataarray = xr.DataArray(np.full((len(ds['year']),len(ds['lat']), len(ds['lon'])), np.nan), 
                #             coords={'lon': ds['lon'], 'lat': ds['lat'],'year': ds['year']}, dims=['year','lat', 'lon'])
 
   
                # Assign the new variable to the dataset
                dsmerge_f[var] = nan_dataarray
            
            #----merge all variables into one dataset----#
            # if it's the first variable, then start a new datset, otherwise merge with existing
            if v ==0:
                dsmerge_v = dsmerge_f.copy()
            else:
                dsmerge_v=xr.merge([dsmerge_v, dsmerge_f])

            # add a new variable that is the sum of all carbon pools
            if all(var_name in dsmerge_v for var_name in ['cVeg', 'cSoil', 'cLitter']):
                if (dsmerge_v['cLitter'].notnull().all()): #litter is sometimes missing. Would be good to make this more general but dealing with this problem for now.
                    dsmerge_v['cTot'] = dsmerge_v['cVeg']+dsmerge_v['cSoil']+dsmerge_v['cLitter'] 
                else: 
                    dsmerge_v['cTot'] = dsmerge_v['cVeg']+dsmerge_v['cSoil'] 
                
        #----save output to a dictionary----#
        print('adding ' +model +' ' +runlist[r] +' to dict')
        data_dict[model +'_' +runlist[r]] = dsmerge_v


In [ ]:
arg=dsmerge_v.soil_carbon_content.mean(dim='time').values
arg

In [ ]:
(dsmerge_f.time.values-dsmerge_v.time.values).plot()


In [ ]:
testnc=xr.open_dataset('/glade/campaign/cgd/tss/people/aswann/flat10/UKESM1.2/flat10/test.nc')

In [ ]:
arg=testnc.soil_carbon_content.mean(dim='time').values
arg

In [ ]:
missing_value = 1.0e36
for var in ds.data_vars:
    ds[var] = ds[var].where(ds[var] < missing_value * 0.1, np.nan)

# Loading CO2

In [ ]:
outputdir= '/glade/campaign/cgd/tss/people/aswann/flat10/'

modellist_orig= ['ACCESS-ESM1-5',  
            'CESM2',    
            'GFDL-ESM4',  
            'GISS_E2.1',  
            'NorESM2-LM',
            'MPI-ESM1-2-LR',
            'CNRM-ESM2-1',
            'HadCM3LC-Bris',
            'UKESM1.2']
modellist=modellist_orig

runlist = ['flat10','flat10_zec','flat10_cdr']
# use a wildcard to capture different ways the folders and runs are named across models
runlist_wc = ['*lat10','*zec','*cdr']

varlist_load=['cVeg','cSoil','cLitter','nbp','gpp','rh','tas','pr'] #, 'gpp','fgco2', 'ra', 'rh']#, 'npp'] # not working beyond nbp for norESM
varlist_analyze=['cVeg','cSoil','cTot','cLitter','nbp','gpp','rh','tas','pr']
varlist=varlist_load
unitslist=['kgC m-2','kgC m-2','kgC m-2','kgC m-2 s-1','kgC m-2 s-1','kgC m-2 s-1','K','kg m-2 s-1']

# there seems to be a problem with ra for NorESM

modelcolors=['tab:blue','tab:orange','tab:green','tab:red','tab:gray','tab:purple','tab:cyan','gold','tab:brown']


In [ ]:
# This function is only for co2 since that is stored in a different place
var='co2'


# data location
outputdir= '/glade/u/home/aswann/work/jupyter_notebooks/esm-deck/ESM_data/'

#----loop over models----#
for m in range(len(modellist)):
#for m in range(len(['GFDL-ESM4',  'GISS_E2.1',  'NorESM2-LM','MPI-ESM1-2-LR'])):
    model=modellist[m]
    print('loading model: ' +model)
    #----loop over experiments----# 
    for r in range(len(runlist)):
        run = runlist_wc[r]
        print('loading run: ' +run)

        print('loading variable: ' +var)
        
        searchpath= outputdir +model +'/' +run +'_' +var +'_*.nc'
        #if ((model =='CESM2') or (model == 'UKESM1.2')):
        #    # all models have a var_ filename except CESM
        #    searchpath= outputdir +model +'/' +run +'*_' +var +'*.nc'
        
        filenamelist= np.sort(glob.glob(searchpath)) # sort in time order, xarray was having trouble arranging some of them in time dim

        print(filenamelist)


In [ ]:
dataco2=pd.read_csv('/glade/work/aswann/jupyter_notebooks/esm-deck/output/atmospheric_co2_flat10.csv')

In [ ]:
dataco2

In [ ]:
dataco2.plot()

# Load global mean data using ben sanderson's esm-deck code for the flat10MIP paper

In [ ]:
import os
import matplotlib.patches as patches
from scipy.stats import linregress

bsdataloc='/glade/work/aswann/jupyter_notebooks/esm-deck/'


In [ ]:
# load emissions data

e_flat10=pd.read_csv(bsdataloc +'flat10.csv',header=4)
e_flat10_cdr=pd.read_csv(bsdataloc +'flat10_cdr.csv',header=4)
e_flat10_zec=pd.read_csv(bsdataloc +'flat10_zec.csv',header=4)
e_flat10_nz=pd.read_csv(bsdataloc +'flat10_cdr.csv',header=4)
e_flat10_nz[150:]=0

In [ ]:



# ESM data
def extd(ts,leng):
    out=np.zeros(leng)
    if leng>len(ts):
        out[:len(ts)]=ts.values
        out[len(ts):]=ts.iloc[-1]
    else:
        out=ts[:leng]
    return out
#mdls=['ACCESS-ESM1-5','CESM2','GFDL-ESM4','GISS','NorESM2-LM','CNRM-ESM2-1','MPI-ESM1-2-LR']
mdls=['ACCESS-ESM1-5','CESM2','GFDL-ESM4','GISS','NorESM2-LM','MPI-ESM1-2-LR','CNRM-ESM2-1','UKESM','UVic-ESCM-2-10','HadCM3LC-Bris']
expt=['esm-pictrl','1pctco2','flat10','flat10-zec','flat10-cdr']
flds=['tas','co2','fgco2','nbp','fCmissing']

ed={}
for m in mdls:
    ed[m]={}
    for e in expt:
        tmpa=[]
        for f in flds:
                print(f)
                fl10=bsdataloc +'ESM_data/'+m+'/'+'flat10'+'_'+f+'_'+m+'.nc'
                fl=bsdataloc +'ESM_data/'+m+'/'+e+'_'+f+'_'+m+'.nc'
                if os.path.exists(fl) and os.path.exists(fl10):
                    otmp=xr.open_dataset(fl)
                    otmp10=xr.open_dataset(fl10)
                    if 'nep' in list(otmp.keys()):
                        otmp=otmp.rename_vars({"nep": "nbp"})
                    if 'nep' in list(otmp10.keys()):
                        otmp10=otmp10.rename_vars({"nep": "nbp"})
                    if 'CO2_GLOB' in list(otmp10.keys()):
                        otmp10=otmp10.rename_vars({"CO2_GLOB": "co2"})
                        otmp=otmp.rename_vars({"CO2_GLOB": "co2"})
                    if 'NBP_GLOB' in list(otmp10.keys()):
                        otmp10=otmp10.rename_vars({"NBP_GLOB": "nbp"})
                        otmp=otmp.rename_vars({"NBP_GLOB": "nbp"})
                    if 'FGCO2_GLOB' in list(otmp10.keys()):
                        otmp10=otmp10.rename_vars({"FGCO2_GLOB": "fgco2"})
                        otmp=otmp.rename_vars({"FGCO2_GLOB": "fgco2"})    
                    if 'TIME' in list(otmp10.coords.keys()):                  
                        otmp10=otmp10.swap_dims({"TIME": "time"}).rename_vars({"TIME": "time"})
                    if 't' in list(otmp10.coords.keys()):                  
                        otmp10=otmp10.swap_dims({"t": "time"}).rename_vars({"t": "time"})
                    if 't' in list(otmp.coords.keys()):                  
                        otmp=otmp.swap_dims({"t": "time"}).rename_vars({"t": "time"})
                    if 'TIME' in list(otmp.coords.keys()):                  
                        otmp=otmp.swap_dims({"TIME": "time"}).rename_vars({"TIME": "time"})
                    if 'TAS_GLOB' in list(otmp10.keys()):                  
                        otmp10=otmp10.rename_vars({"TAS_GLOB": "tas"})
                    if 'TAS_GLOB' in list(otmp.keys()):                  
                        otmp=otmp.rename_vars({"TAS_GLOB": "tas"})
                        
                    if 'temp_mm_1_5m' in list(otmp10.keys()):                  
                        otmp10=otmp10.rename_vars({"temp_mm_1_5m": "tas"})
                    if 'temp_mm_1_5m' in list(otmp.keys()):                  
                        otmp=otmp.rename_vars({"temp_mm_1_5m": "tas"})
                    
                    if 'field1564_mm_hyb' in list(otmp10.keys()):
                        otmp10=otmp10.rename_vars({"field1564_mm_hyb": "co2"})
                        otmp=otmp.rename_vars({"field1564_mm_hyb": "co2"})
                    if 'unknown' in list(otmp10.keys()):
                        otmp10=otmp10.rename_vars({"unknown": "nbp"})
                        otmp=otmp.rename_vars({"unknown": "nbp"})
                   
                    if 'field1560_mm_srf' in list(otmp10.keys()):
                        otmp10=otmp10.rename_vars({"field1560_mm_srf": "fgco2"})
                        otmp=otmp.rename_vars({"field1560_mm_srf": "fgco2"})
                    if 'FCMISSING' in list(otmp.keys()):                  
                        otmp=otmp.rename_vars({"FCMISSING": "fCmissing"})    
                    if 'FCMISSING' in list(otmp10.keys()):                  
                        otmp10=otmp10.rename_vars({"FCMISSING": "fCmissing"})      
                    if '__xarray_dataarray_variable__' in list(otmp10.keys()):                  
                        otmp10=otmp10.rename_vars({"__xarray_dataarray_variable__": f})
                    if '__xarray_dataarray_variable__' in list(otmp.keys()):                  
                        otmp=otmp.rename_vars({"__xarray_dataarray_variable__": f})

                    tmp=otmp.copy(deep=True)
                    tmp10=otmp10.copy(deep=True)
                    if len(tmp.time) > 4800:
                        tmp = tmp.isel(time=slice(0, 4800))
                    if len(tmp10.time) > 4800:
                        tmp10 = tmp10.isel(time=slice(0, 4800))
                    if e=='flat10':
                        if len(tmp.time)<1000:
                            tmp.coords['time']=pd.date_range('1700-01-01', periods=tmp['time'].values.shape[0],freq='Y')
                        else:
                            tmp.coords['time']=pd.date_range('1700-01-01', periods=tmp['time'].values.shape[0],freq='M')
                        tmp=tmp[f]    
                    else: 
                        if len(tmp.time)<1000:
                            tmp10.coords['time']=pd.date_range('1700-01-01', periods=tmp10['time'].values.shape[0],freq='Y')
                            tmp.coords['time']=pd.date_range('1800-01-01', periods=tmp['time'].values.shape[0],freq='Y')
                        else:
                            tmp10.coords['time']=pd.date_range('1700-01-01', periods=tmp10['time'].values.shape[0],freq='M')
                            tmp.coords['time']=pd.date_range('1800-01-01', periods=tmp['time'].values.shape[0],freq='M')
  
                        tmp=xr.concat((tmp10.where(tmp10['time.year'] < 1800, drop=True)[f],tmp[f]),'time')
                    tmp=tmp.groupby('time.year').mean()
                    if len(tmp.shape)==2:
                        tmp=tmp.sel(lat=0).sel(lon=0).drop('lev')
                    if len(tmp.shape)==3:
                        tmp=tmp.sel(lat=0).sel(lon=0).drop('lat').drop('lon')
                    if len(tmp.shape)==4:
                        if 'lev' in list(otmp10.coords.keys()):
                            tmp=tmp.sel(lat=0).sel(lon=0).isel(lev=[0]).drop('lat').drop('lon').drop('lev')
                        else:
                            tmp=tmp.sel(lat=0).sel(lon=0).isel(plev=[0]).drop('lat').drop('lon').drop('plev')
                    
                    if tmp.name=='tas':            
                        t10=tmp[0:20].mean().values
                        d10=tmp[20:40].mean().values-t10
                        t0=t10-d10/2
                        tmp=tmp-t0
                    tmpa.append(tmp)
                    if f=='fgco2' and m=='NorESM2-LM' and e=='flat10':
                     0
                else:
                    print(m+e+f)
                
             
        ed[m][e]=xr.merge(tmpa)
        if len(tmpa)>0:
            ed[m][e]['fco2fos']=ed[m][e]['tas'].copy()
            if e=='flat10':
                    ed[m][e]['fco2fos'].values=extd(e_flat10['Emission Rate (PgC a-1)'],len(ed[m][e]['year']))
            if e=='flat10-cdr':
                    ed[m][e]['fco2fos'].values=extd(e_flat10_cdr['Emission Rate (PgC a-1)'],len(ed[m][e]['year']))
            if e=='flat10-zec':
                    ed[m][e]['fco2fos'].values=extd(e_flat10_zec['Emission Rate (PgC a-1)'],len(ed[m][e]['year']))
            if m=='ACCESS-ESM1-5':
                ed[m][e]['co2']=ed[m][e]['co2']*1e6

            if m=='GFDL-ESM4':
                ed[m][e]['co2']=ed[m][e]['co2'].isel(lev=0)*1e6
                #ed[m][e]['nbp']=ed[m][e]['nbp']*5.1e14/1e12*3600*24*365
                aco2=np.diff(ed[m][e]['co2'][:],append=ed[m][e]['co2'][-1])*2.12
                ln=len(ed[m][e]['fco2fos'])
                ed[m][e]['fgco2']=ed[m][e]['fgco2'][:ln]
                ed[m][e]['nbp']=ed[m][e]['fco2fos'][:ln]-aco2[:ln]-ed[m][e]['fgco2'][:ln]

            if m=='CESM2':
                ed[m][e]['fgco2']=ed[m][e]['fgco2']/1e15
                ed[m][e]['nbp']=ed[m][e]['nbp']/1e15
                
                ed[m][e]['co2']=280+np.cumsum(ed[m][e]['fco2fos']-ed[m][e]['fgco2']-ed[m][e]['nbp'])/2.12
            
                ed[m][e]['co2']=280+np.cumsum(ed[m][e]['fco2fos']-ed[m][e]['fgco2']-ed[m][e]['nbp'])/2.12
            if m=='NorESM2-LM':
                ed[m][e]['co2']=ed[m][e]['co2']*1e6
                #ed[m][e]['nbp']=ed[m][e]['nbp']#*5.1e14/1e12*3600*24*365
                aco2=np.diff(ed[m][e]['co2'][:],append=ed[m][e]['co2'][-1])*2.12
                ed[m][e]['nbp']=ed[m][e]['fco2fos']-aco2-ed[m][e]['fgco2']
                #ed[m][e]['fgco2']=ed[m][e]['fgco2']*5.1e14/1e12*3600*24*365
            if m=='UKESM':
                ed[m][e]['nbp']=ed[m][e]['nbp']*5.1e14/1e12*.29
                #ed[m][e]['nbp']=ed[m][e]['nbp']*5.1e14/1e12*3600*24*365
                #ed[m][e]['fgco2']=ed[m][e]['fgco2']*5.1e14/1e12*3600*24*365*.71
            if m=='CNRM-ESM2-1':
                ed[m][e]['fgco2']=ed[m][e]['fgco2']-ed[m][e]['fCmissing']
                #ed[m][e]['nbp']=ed[m][e]['nbp']*5.1e14/1e12*3600*24*365
                #ed[m][e]['fgco2']=ed[m][e]['fgco2']*5.1e14/1e12*3600*24*365*.71
            if m=='HadCM3LC-Bris':
                0#ed[m][e]['fgco2']=ed[m][e]['fgco2']*5.1e14/1e12*.71
                ed[m][e]['nbp']=ed[m][e]['nbp']
                


ed["GFDL-ESM4"]['flat10']['fgco2'][0] = ed["GFDL-ESM4"]['flat10']['fgco2'][0]-10        # correct first ocean value of GFDL-ESM4
ed["GFDL-ESM4"]['flat10-zec']['fgco2'][0] = ed["GFDL-ESM4"]['flat10-zec']['fgco2'][0]-10        # correct first ocean value of GFDL-ESM4
ed["GFDL-ESM4"]['flat10-cdr']['fgco2'][0] = ed["GFDL-ESM4"]['flat10-cdr']['fgco2'][0]-10        # correct first ocean value of GFDL-ESM4

for i,m in enumerate(mdls):
    print(m)
    try:
        ed[m]['flat10-cdr']['C_atm']=(ed[m]['flat10-cdr']['co2']-ed[m]['flat10']['co2'][0])*2.13
    except:
        print('fail cdr')
    try:
        ed[m]['flat10-zec']['C_atm']=(ed[m]['flat10-zec']['co2']-ed[m]['flat10']['co2'][0])*2.13
    except:
        print('fail zec')
    try:
        ed[m]['flat10']['C_atm']=(ed[m]['flat10']['co2']-ed[m]['flat10']['co2'][0])*2.13

    except:
        print('fail flat')    


t0mat = {}
for i, m in enumerate(mdls):
    # Extract the first 30 years of the temperature timeseries
    x = np.arange(0, 30)  # Years 0 to 29
    y = np.array(ed[m]['flat10']['tas'][:30])  # Corresponding temperature values

    # Perform linear regression to estimate temperature at time zero
    slope, intercept, _, _, _ = linregress(x, y)
    t0mat[m] = intercept  # Use the intercept as the temperature at time zero

    # Adjust the temperature timeseries for each experiment
    ed[m]['flat10-cdr']['tas'] = ed[m]['flat10-cdr']['tas'] - intercept
    ed[m]['flat10-zec']['tas'] = ed[m]['flat10-zec']['tas'] - intercept
    ed[m]['flat10']['tas'] = ed[m]['flat10']['tas'] - intercept

In [ ]:
ed

In [ ]:
#can save to csv files like this

# Initialize empty dictionaries to store data for each model
data_a = {}
data_d = {}
data_g = {}
data_j = {}

# Iterate over models and dynamically handle timeseries lengths
for model in mdls:
    # Get the length of the timeseries for the current model
    length = len(ed[model]['flat10']['tas'])

    # Extract data for the available length
    data_a[model] = ed[model]['flat10']['tas'].values[:length]
    data_d[model] = ed[model]['flat10']['co2'].values[:length]
    data_g[model] = np.cumsum(ed[model]['flat10']['nbp'].values[:length])
    data_j[model] = np.cumsum(ed[model]['flat10']['fgco2'].values[:length])

# Convert dictionaries to DataFrames
df_a = pd.DataFrame.from_dict(data_a, orient='index').transpose()
df_d = pd.DataFrame.from_dict(data_d, orient='index').transpose()
df_g = pd.DataFrame.from_dict(data_g, orient='index').transpose()
df_j = pd.DataFrame.from_dict(data_j, orient='index').transpose()

# Save the DataFrames to CSV files
df_a.to_csv("output/temperatures_flat10.csv", index=False)
df_d.to_csv("output/atmospheric_co2_flat10.csv", index=False)
df_g.to_csv("output/land_carbon_sink_flat10.csv", index=False)
df_j.to_csv("output/ocean_carbon_sink_flat10.csv", index=False)

In [4]:
# ## Create a netcdf file with a matrix of processed time series
# this works with environment npl2025b
# to run on the command line:
#
# module load conda
# conda activate npl-2025b
# python create_metrics_matrix.py


# This is a script version of a python notebook


import numpy as np
import numpy.matlib
import numpy.ma as ma

import xarray as xr


import time
import cftime
import netCDF4 as nc
from datetime import timedelta

import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

import glob


import nc_time_axis
import cartopy.crs as ccrs
import cartopy


import warnings
#warnings.filterwarnings("ignore", message="invalid value encountered in divide")

# --- custom files
# load custom functions for analyzing flat10

from loading_function_flat10 import load_flat10, load_one_model, load_one_model_onevar, load_grid, select_time_slice, weighted_temporal_mean 


# ###-------------####
# define which runs and models

outputdir= '/glade/campaign/cgd/tss/people/aswann/flat10/'

modellist_orig= ['ACCESS-ESM1-5',  
            'CESM2',    
            'GFDL-ESM4',  
            'GISS_E2.1',  
            'NorESM2-LM',
            'MPI-ESM1-2-LR',
            'CNRM-ESM2-1',
            'HadCM3LC-Bris',
            'UKESM1.2']
modellist=modellist_orig

runlist = ['flat10','flat10_zec','flat10_cdr']
# use a wildcard to capture different ways the folders and runs are named across models
runlist_wc = ['*lat10','*zec','*cdr']

varlist_load=['cVeg','cSoil','cLitter','nbp','gpp','rh','tas','pr'] #, 'gpp','fgco2', 'ra', 'rh']#, 'npp'] # not working beyond nbp for norESM
varlist_analyze=['cVeg','cSoil','cTot','cLitter','nbp','gpp','rh','tas','pr']
varlist=varlist_load
unitslist=['kgC m-2','kgC m-2','kgC m-2','kgC m-2 s-1','kgC m-2 s-1','kgC m-2 s-1','K','kg m-2 s-1']


# global and three latitude bands: trop, mid, high
latlist=['global','highlat','troplat','midlat']
troplat=30
highlat=60

## unit conversions
#unitconversions
sperday=60*60*24
speryr=60*60*24*365

PgperKg = 1e-12
# to convert kg m-2 s-1 to kg/m2/yr multiply by speryr

# initialize a dictionary to hold all of the data
data_dict={}


# ###---------------####

#-- load grid info
data_dict = load_grid(data_dict,modellist)


#-- load data
#data_dict=load_flat10(data_dict, modellist, runlist, runlist_wc, varlist)

#-- toggle to analysis lists
modellist=modellist_orig
varlist=varlist_load


# ###---------------####
# create matrix of zonal mean time series for 

#- initialize
C_global_mat= np.empty([350,len(modellist),len(runlist),len(varlist)])
C_highlat_mat= np.empty([350,len(modellist),len(runlist),len(varlist)])
C_troplat_mat= np.empty([350,len(modellist),len(runlist),len(varlist)])
C_midlat_mat= np.empty([350,len(modellist),len(runlist),len(varlist)])

# create a time series of years for the first dimension
ts= np.arange(350)

for m in range(len(modellist)):
#for m in range(len(['GFDL-ESM4',  'GISS_E2.1',  'NorESM2-LM','MPI-ESM1-2-LR'])):
    model=modellist[m]
    print('processing model: ' +model)

    # get area and landfrac from the dictionary where they have been pre-loaded
    ds_area = data_dict[modellist[m] +'_' +'areacella']
    ds_landfrac = data_dict[modellist[m] +'_' +'landfrac']
    
    #----loop over experiments----# 
    for e in range(len(runlist)):
        run = runlist[e]
        print('processing run: ' +run)

        #ds=load_one_model(model,runlist_wc[e],varlist)
        #ds=data_dict[modellist[m] +'_' +runlist[e]]


        #----loop over variables----#
        for v in range(len(varlist)):
            var=varlist[v]
            print('processing variable: ' +var)

            ds=load_one_model_onevar(model,runlist_wc[e],var)


            if model=='CESM2':
                area = ds_area['areacella'].squeeze().reindex_like(ds, method='nearest',tolerance=0.05)
            else:
                area = ds_area['areacella'].reindex_like(ds, method='nearest',tolerance=0.05)
            
            landfrac=ds_landfrac['sftlf'].reindex_like(ds, method='nearest',tolerance=0.05)
            
            if landfrac.max(dim=['lat','lon'])>1: #test if landfrac is on a 0-100 or 0-1 scale
                landfrac=landfrac/100
                
            landarea=area*landfrac
             

            # NorESM has drift that needs to be corrected
            # load the drift correction matrix and remove the drift
            if model=='NorESM2-LM':
                if var=='cVeg':
                    field = pickle.load(open('/glade/campaign/cgd/tss/people/aswann/flat10/NorESM2-LM/NorESM2-LM_2D_TOTVEGC_ann_drift.pkl','rb'))
                    adj_matrix = xr.DataArray(np.squeeze(field), dims=['lat','lon'], coords={'latitude': ds.lat, 'longitude':ds.lon})##,unit={'g C m-2 yr-1'})
                    ty=ds['time'].dt.year
                    tyindx=ty-ty[0]+1
                    adjustment = adj_matrix* tyindx*(1/1000) #this is the drift for each time point and each gridcell in kg C m-2 yr-1

                    ds[var]=ds[var]+adjustment # remove the drift from the variable
                    
                elif var=='cSoil':
                    field = pickle.load(open('/glade/campaign/cgd/tss/people/aswann/flat10/NorESM2-LM/NorESM2-LM_2D_TOTSOMC_ann_drift.pkl','rb'))
                    adj_matrix = xr.DataArray(np.squeeze(field), dims=['lat','lon'], coords={'latitude': ds.lat, 'longitude':ds.lon})##,unit={'g C m-2 yr-1'})
                    ty=ds['time'].dt.year
                    tyindx=ty-ty[0]+1
                    adjustment = adj_matrix* tyindx*(1/1000) #this is the drift for each time point and each gridcell in kg C m-2 yr-1

                    ds[var]=ds[var]+adjustment # remove the drift from the variable
                    
                elif var=='cLitter':
                    field = pickle.load(open('/glade/campaign/cgd/tss/people/aswann/flat10/NorESM2-LM/NorESM2-LM_2D_TOTLITC_ann_drift.pkl','rb'))
                    adj_matrix = xr.DataArray(np.squeeze(field), dims=['lat','lon'], coords={'latitude': ds.lat, 'longitude':ds.lon})##,unit={'g C m-2 yr-1'})
                    ty=ds['time'].dt.year
                    tyindx=ty-ty[0]+1
                    adjustment = adj_matrix* tyindx*(1/1000) #this is the drift for each time point and each gridcell in kg C m-2 yr-1

                    ds[var]=ds[var]+adjustment # remove the drift from the variable

            data_var= weighted_temporal_mean(ds, var)

            # mask for nans 
            # Mask landarea where it's zero or NaN to avoid invalid values
            valid_mask = (landarea > 0) & landarea.notnull()
            masked_landarea = landarea.where(valid_mask)
            masked_data = data_var.where(valid_mask)

            landarea_global = masked_landarea.sum(dim=['lat','lon'])
            #landarea_global = landarea_global.where(landarea_global != 0)
            landarea_highlat = ((masked_landarea.where(ds.lat>=highlat)).sum(dim=['lat','lon']))
            #landarea_highlat = landarea_highlat.where(landarea_highlat != 0)
            landarea_troplat = ((masked_landarea.where((ds.lat>=-troplat) & (ds.lat<=troplat))).sum(dim=['lat','lon']))
            #landarea_troplat = landarea_troplat.where(landarea_troplat != 0)
            landarea_midlat = ((masked_landarea.where((ds.lat>=troplat) & (ds.lat<=highlat))).sum(dim=['lat','lon']))
            #landarea_midlat = landarea_midlat.where(landarea_midlat != 0)

            if var=='tas' or var=='pr': 
                # if this is *not* a carbon variable then we want to make an average
                # C_global =(((data_var*landarea)).sum(dim=['lat','lon']))/landarea.sum(dim=['lat','lon'])
                # C_highlat=(((data_var*landarea).where(ds.lat>=highlat)).sum(dim=['lat','lon']))/((landarea.where(ds.lat>=highlat)).sum(dim=['lat','lon']))
                # C_troplat=(((data_var*landarea).where((ds.lat>=-troplat) & (ds.lat<=troplat))).sum(dim=['lat','lon']))/((landarea.where((ds.lat>=-troplat) & (ds.lat<=troplat))).sum(dim=['lat','lon']))
                # C_midlat=(((data_var*landarea).where((ds.lat>=troplat) & (ds.lat<=highlat))).sum(dim=['lat','lon']))/((landarea.where((ds.lat>=troplat) & (ds.lat<=highlat))).sum(dim=['lat','lon']))

                C_global =(((data_var*landarea)).sum(dim=['lat','lon']))/landarea_global
                C_highlat=(((data_var*landarea).where(ds.lat>=highlat)).sum(dim=['lat','lon']))/landarea_highlat
                C_troplat=(((data_var*landarea).where((ds.lat>=-troplat) & (ds.lat<=troplat))).sum(dim=['lat','lon']))/landarea_troplat
                C_midlat=(((data_var*landarea).where((ds.lat>=troplat) & (ds.lat<=highlat))).sum(dim=['lat','lon']))/landarea_midlat
    
                #put into matrix 
                C_global_mat[0:len(C_global),m,e,v]= C_global
                C_highlat_mat[0:len(C_global),m,e,v]= C_highlat
                C_troplat_mat[0:len(C_global),m,e,v]= C_troplat
                C_midlat_mat[0:len(C_global),m,e,v]= C_midlat
            
            else: # it is a carbon variable and we want to make a sum
                # total carbon on land. Becuase it is in units of carbon/area (kgC/m2), multiply by area
                # our area variable is in m2
                C_global =(((data_var*landarea)).sum(dim=['lat','lon']))
                C_highlat=((data_var*landarea).where(ds.lat>=highlat)).sum(dim=['lat','lon'])
                C_troplat=((data_var*landarea).where((ds.lat>=-troplat) & (ds.lat<=troplat))).sum(dim=['lat','lon'])
                C_midlat=((data_var*landarea).where((ds.lat>=troplat) & (ds.lat<=highlat))).sum(dim=['lat','lon'])
    
                #put into matrix and convert to PgC (kgC => PgC, divide by 10^12)
                C_global_mat[0:len(C_global),m,e,v]= C_global*PgperKg
                C_highlat_mat[0:len(C_global),m,e,v]= C_highlat*PgperKg
                C_troplat_mat[0:len(C_global),m,e,v]= C_troplat*PgperKg
                C_midlat_mat[0:len(C_global),m,e,v]= C_midlat*PgperKg

            # reset values after the end of the time series to nan
            C_global_mat[(len(C_global)):,m,e,v]=np.nan
            C_highlat_mat[(len(C_highlat)):,m,e,v]=np.nan
            C_troplat_mat[(len(C_troplat)):,m,e,v]=np.nan
            C_midlat_mat[(len(C_midlat)):,m,e,v]=np.nan

        del ds # remove the dataset from memory
        del data_var # remove from memory

# ###----------------####

# put the matrix into an xarray dataset
data_array_combined = np.full((len(ts), len(modellist), len(runlist), len(varlist), len(latlist)),np.nan)

data_array_combined[:,:,:,:,0]=C_global_mat
data_array_combined[:,:,:,:,1]=C_highlat_mat
data_array_combined[:,:,:,:,2]=C_troplat_mat
data_array_combined[:,:,:,:,3]=C_midlat_mat


# ###----------------####
# put into an xarray dataset

ds_C_global= xr.Dataset(
    {
        "data": (["time", "model", "run", "var","latrange"], data_array_combined)
    },
    coords={
        "time": ts,
        "model": modellist,
        "run": runlist,
        "var": varlist,
        "latrange": latlist
    }
)


# ###----------------####

# - save the matrix to a netcdf file
##ds_C_global.to_netcdf("C_metrics_matrix.nc")


ACCESS-ESM1-5 getting grid info
CESM2 getting grid info
GFDL-ESM4 getting grid info
GISS_E2.1 getting grid info
NorESM2-LM getting grid info
MPI-ESM1-2-LR getting grid info
CNRM-ESM2-1 getting grid info
HadCM3LC-Bris getting grid info
UKESM1.2 getting grid info
processing model: ACCESS-ESM1-5
processing run: flat10
processing variable: cVeg
loading variable: cVeg
finished loading ACCESS-ESM1-5 *lat10 cVeg
processing variable: cSoil
loading variable: cSoil
finished loading ACCESS-ESM1-5 *lat10 cSoil
processing variable: cLitter
loading variable: cLitter
finished loading ACCESS-ESM1-5 *lat10 cLitter
processing variable: nbp
loading variable: nbp
finished loading ACCESS-ESM1-5 *lat10 nbp
processing variable: gpp
loading variable: gpp
finished loading ACCESS-ESM1-5 *lat10 gpp
processing variable: rh
loading variable: rh
finished loading ACCESS-ESM1-5 *lat10 rh
processing variable: tas
loading variable: tas
finished loading ACCESS-ESM1-5 *lat10 tas
processing variable: pr
loading variable: 

KeyboardInterrupt: 

In [2]:
searchpath= outputdir +model +'/' +run +'/*' +var +'_*.nc'
searchpath

'/glade/campaign/cgd/tss/people/aswann/flat10/ACCESS-ESM1-5/flat10_cdr/*cLitter_*.nc'

In [3]:
filenamelist= np.sort(glob.glob(searchpath))
filenamelist

array([], dtype=float64)